In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
def makeABIrgb_fromReflectance(R_ref, G_ref, B_ref):
    
    ''' Create RGB images given GOES-R ABI Channel 01, 02, and 03 datasets.
        Adapted from https://github.com/daniellelosos/True-Color-Image_GOES-R-ABI-L1b-Radiances
        
        returns:
                RGB: "True Color" RGB
                RGB_veggie: "False Color" RGB
    '''
    
    # Apply range limits for each channel. Reflectance values must be between 0 and 1.
    R_ref = (R_ref - np.min(R_ref)) / (np.max(R_ref) - np.min(R_ref)) # np.clip(R_ref, 0, 1)
    G_ref = (G_ref - np.min(G_ref)) / (np.max(G_ref) - np.min(G_ref)) # np.clip(G_ref, 0, 1)
    B_ref = (B_ref - np.min(B_ref)) / (np.max(B_ref) - np.min(B_ref)) # np.clip(B_ref, 0, 1)
    
    # Apply a gamma correction to the image to correct ABI detector brightness
    gamma = 2.2
    Red = np.power(R_ref, 1/gamma)
    Green = np.power(G_ref, 1/gamma)
    Blue = np.power(B_ref, 1/gamma)
    
    # GOES-R Series satellites do not have a channel in the visible green range. Band 3 is a NIR channel typically used to monitor vegetation.
    # Calculate the "True" Green Band to serve as a green proxy for the RGB True Color image, using a fractional combination.
    # Source: "Generation of GOES‐16 True Color Imagery without a Green Band" - https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2018EA000379
    Green_true = 0.45 * Red + 0.1 * Green + 0.45 * Blue
    Green_true = np.clip(Green_true, 0, 1)  # Apply band limits again, just in case.
    
    # Combine three RGB channels with a stacked array, then display the resulting images.
    
    # The RGB array with the raw veggie band
    RGB_veggie = np.stack([Red, Green, Blue],axis=3)
    
    # The RGB array for the true color image
    RGB = np.stack([Red, Green_true, Blue],axis=3)
    
    return RGB, RGB_veggie

In [3]:
ds16b1 = xr.open_zarr('goes16-build-zarr-b1.zarr')
ds18b1 = xr.open_zarr('goes18-build-zarr-b1.zarr')

ds16b2 = xr.open_zarr('goes16-build-zarr-b2.zarr')
ds18b2 = xr.open_zarr('goes18-build-zarr-b2.zarr')

ds16b3 = xr.open_zarr('goes16-build-zarr-b3.zarr')
ds18b3 = xr.open_zarr('goes18-build-zarr-b3.zarr')

In [4]:
ds16b1['time'] = ds18b1.time
ds18b1['Rad'] = (ds16b1.Rad + ds18b1.Rad) / 2
da_b1 = ds18b1.Rad

ds16b2['time'] = ds18b2.time
ds18b2['Rad'] = (ds16b2.Rad + ds18b2.Rad) / 2
da_b2 = ds18b2.Rad

ds16b3['time'] = ds18b3.time
ds18b3['Rad'] = (ds16b3.Rad + ds18b3.Rad) / 2
da_b3 = ds18b3.Rad



In [5]:
goes_RGB, goes_RGB_veggie = makeABIrgb_fromReflectance(da_b2, da_b3, da_b1)

In [6]:
def scale_values(img, m=255):
        img_scaled = (img - np.min(img)) / (np.max(img) - np.min(img))
        return (img_scaled * m).astype(np.uint8)

In [7]:
imgs = np.random.randint(0, 255, (100, 50, 50, 3), dtype=np.uint8)
imgs = [Image.fromarray(scale_values(img)) for img in goes_RGB]
# duration is the number of milliseconds between frames; this is 40 frames per second
imgs[0].save("array2.gif", save_all=True, append_images=imgs[1:], duration=100, loop=0)